In [1]:
import sys
import os
import tf
import cv2
import rospy
import rosbag
import progressbar
from tf2_msgs.msg import TFMessage
from datetime import datetime
from std_msgs.msg import Header
from sensor_msgs.msg import CameraInfo, Imu, PointField, NavSatFix
import sensor_msgs.point_cloud2 as pcl2
from geometry_msgs.msg import TransformStamped, TwistStamped, Transform, PoseStamped
from cv_bridge import CvBridge
import numpy as np
import argparse

In [2]:
try:
    import pykitti
except ImportError as e:
    print('Could not load module \'pykitti\'. Please run `pip install pykitti`')
    sys.exit(1)

In [3]:
def save_velo_data(bag, dpath, topic, max_num=500):
    print("Exporting velodyne data")
    velo_filenames = sorted(os.listdir(dpath))[:max_num]
    times = np.arange(len(velo_filenames)) * 0.1

    iterable = zip(times, velo_filenames)
    bar = progressbar.ProgressBar()
    i = 0
    for dt, filename in bar(iterable):
        velo_filename = os.path.join(dpath, filename)

        # read binary data
        scan = (np.fromfile(velo_filename, dtype=np.float32)).reshape(-1, 4)

        # create header
        header = Header()
        header.frame_id = "velo_link"
        header.stamp = rospy.Time.from_sec(times[i])

        # fill pcl msg
        fields = [PointField('x', 0, PointField.FLOAT32, 1),
                  PointField('y', 4, PointField.FLOAT32, 1),
                  PointField('z', 8, PointField.FLOAT32, 1),
                  PointField('i', 12, PointField.FLOAT32, 1)]
        pcl_msg = pcl2.create_cloud(header, fields, scan)

        bag.write('/map_pointcloud', pcl_msg, t=pcl_msg.header.stamp)
        i += 1

In [10]:
def save_transforms(bag, dpath, topic, max_num=500, kitti=False):
    fpath = os.path.join(dpath, "poses.txt")
    poses = np.loadtxt(fpath)
    
    if kitti:
        Tr_file = np.genfromtxt(os.path.join(dpath, 'calib.txt'))[-1, 1:]
        Tr = np.zeros((4, 4))
        Tr[3, 3] = 1
        Tr[:3, :4] = Tr_file.reshape(3,4)
        Tr = Tr.astype(np.float32)
    
    for i in range(max_num):
        pose = np.zeros((4, 4))
        pose[3, 3] = 1
        pose[:3, :4] = poses[i,:].reshape(3, 4)
        
        if kitti:
            pose = pose.astype(np.float32)
            pose = np.matmul(np.linalg.inv(Tr), np.matmul(pose, Tr))
            
        
        timestamp = 0.1 * i
        
        tf_stamped = PoseStamped()
        tf_stamped.header.stamp = rospy.Time.from_sec(timestamp)
        tf_stamped.header.frame_id = 'world'

        t = pose[0:3, 3]
        q = tf.transformations.quaternion_from_matrix(pose)

        tf_stamped.pose.position.x = t[0]
        tf_stamped.pose.position.y = t[1]
        tf_stamped.pose.position.z = t[2]

        tf_stamped.pose.orientation.x = q[0]
        tf_stamped.pose.orientation.y = q[1]
        tf_stamped.pose.orientation.z = q[2]
        tf_stamped.pose.orientation.w = q[3]

        bag.write('/map_pose', tf_stamped, t=tf_stamped.header.stamp)
    

In [15]:
def run_kitti2bag():
    compression = rosbag.Compression.NONE
    # bag = rosbag.Bag("kitti.bag", 'w')
    bag = rosbag.Bag("rellis.bag", 'w')

    # path = "/home/tigeriv/Data/KITTI/dataset/04/"
    path = "/home/tigeriv/Data/Rellis-3D/Val/00003/"
    
    save_transforms(bag, path, "/rellis")
    # save_transforms(bag, path, "/kitti", kitti=True)
    
    save_velo_data(bag, os.path.join(path, "os1_cloud_node_kitti_bin/"), "/rellis")
    # save_velo_data(bag, os.path.join(path, "velodyne/"), "/kitti")


    print("## OVERVIEW ##")
    print(bag)
    bag.close()

In [16]:
run_kitti2bag()

- | #                                                 | 1 Elapsed Time: 0:00:00

Exporting velodyne data


| |                           #                     | 499 Elapsed Time: 0:01:05


## OVERVIEW ##
path:        rellis.bag
version:     2.0
duration:    49.9s
start:       Dec 31 1969 19:00:00.00 (0.00)
end:         Dec 31 1969 19:00:49.90 (49.90)
size:        1000.2 MB
messages:    1000
compression: none [500/500 chunks]
types:       geometry_msgs/PoseStamped [d3812c3cbc69362b77dc0b19b345f8f5]
             sensor_msgs/PointCloud2   [1158d486dd51d683ce2f1be655c3c181]
topics:      /map_pointcloud   500 msgs    : sensor_msgs/PointCloud2  
             /map_pose         500 msgs    : geometry_msgs/PoseStamped
